# 1. Retreiving data from the Aalto Open Courses API

In [7]:
import os
from dotenv import load_dotenv
import json

load_dotenv()

api_key = os.getenv('COURSE_API_KEY')


In [12]:
import requests

# TODO if data exist no loading
# call API
url = 'https://course.api.aalto.fi:443/api/sisu/v1/courseunitrealisations?user_key=' + api_key
r = requests.get(url)

# get data
raw_data = r.json()

There are 2316 courses in total.
There are 1405 courses in English.
There are 911 courses not in English.


In [ ]:
# remove the courses that have languageOfInstruction = 'en'
print("There are", len(raw_data), "courses in total.")
en_data = [course for course in raw_data if 'en' in course['languageOfInstructionCodes']]
not_en_data = [course for course in raw_data if 'en' not in course['languageOfInstructionCodes']]
print("There are", len(en_data), "courses in English.")
print("There are", len(not_en_data), "courses not in English.")

# save data in a json file
with open('data/data.json', 'w') as f:
    json.dump(en_data, f)

In [10]:
import pandas as pd

courses = pd.DataFrame(data)
courses.head()

,id,code,startDate,endDate,type,name,summary,organizations,organizationId,organizationName,credits,courseUnitId,languageOfInstructionCodes,teachers,enrolmentStartDate,enrolmentEndDate,mincredits,studySubGroups
0,aalto-CUR-162063-3082770,LC-1117,2023-01-11,2023-02-15,teaching-participation-lectures,"{'fi': 'Integrated Oral and Written Skills (o,...",{'workload': {'fi': '81 h (Small group instruc...,[{'organisationId': 'aalto-52ed67c2-4791-4ee6-...,U926,"{'fi': 'Aalto University, Kielikeskus', 'sv': ...","{'min': 3, 'max': 3}",aalto-OPINKOHD-1117673055-20210801,[en],[Hanna Liisa Hakala],2022-12-12,2023-01-02,3,"[{'id': 'aalto-ssg-162063-3262-7-11111', 'name..."
1,aalto-CUR-162064-3082771,LC-1117,2023-01-11,2023-03-29,teaching-participation-lectures,"{'fi': 'Integrated Oral and Written Skills (o,...",{'workload': {'fi': '81 h (Small group instruc...,[{'organisationId': 'aalto-52ed67c2-4791-4ee6-...,U926,"{'fi': 'Aalto University, Kielikeskus', 'sv': ...","{'min': 3, 'max': 3}",aalto-OPINKOHD-1117673055-20210801,[en],[Jenni Maria Korvala],2022-12-12,2023-01-10,3,"[{'id': 'aalto-ssg-162064-3262-7-11111', 'name..."
2,aalto-CUR-162065-3082772,LC-1117,2023-01-17,2023-04-04,teaching-participation-lectures,"{'fi': 'Integrated Oral and Written Skills (o,...",{'workload': {'fi': '81 h (Small group instruc...,[{'organisationId': 'aalto-52ed67c2-4791-4ee6-...,U926,"{'fi': 'Aalto University, Kielikeskus', 'sv': ...","{'min': 3, 'max': 3}",aalto-OPINKOHD-1117673055-20210801,[en],[Maxi-Ann Marie A Campbell],2022-12-12,2023-01-12,3,"[{'id': 'aalto-ssg-3375-162065-7-01', 'name': ..."
3,aalto-CUR-162066-3082773,LC-1117,2023-03-03,2023-05-19,teaching-participation-lectures,"{'fi': 'Integrated Oral and Written Skills (o,...",{'workload': {'fi': '81 h (Small group instruc...,[{'organisationId': 'aalto-52ed67c2-4791-4ee6-...,U926,"{'fi': 'Aalto University, Kielikeskus', 'sv': ...","{'min': 3, 'max': 3}",aalto-OPINKOHD-1117673055-20210801,[en],[Riina Marketta Seppälä],2023-01-30,2023-02-20,3,"[{'id': 'aalto-ssg-162066-3262-7-11111', 'name..."
4,aalto-CUR-162068-3082775,LC-1117,2023-04-25,2023-05-23,teaching-participation-lectures,"{'fi': 'Integrated Oral and Written Skills (o,...",{'workload': {'fi': '81 h (Small group instruc...,[{'organisationId': 'aalto-52ed67c2-4791-4ee6-...,U926,"{'fi': 'Aalto University, Kielikeskus', 'sv': ...","{'min': 3, 'max': 3}",aalto-OPINKOHD-1117673055-20210801,[en],"[Susan Katariina Gamache, Malachy James Halpin]",2023-03-27,2023-04-17,3,"[{'id': 'aalto-ssg-162068-3262-7-11111', 'name..."


In [67]:
# number of courses with unique code
# print(len(courses["code"].unique())) # 980

# number of courses with unique courseUnitId
# print(len(courses["courseUnitId"].unique())) # 988

# measure difference between two measures
unique_courseUnitId = courses["courseUnitId"].unique()
unique_code = courses["code"].unique()

unique_courseUnitId = courses.drop_duplicates(subset=["courseUnitId"], keep="first")
unique_code = courses.drop_duplicates(subset=["code"], keep="first")
# difference of ids
id_difference = list(set(unique_courseUnitId["id"]) - set(unique_code["id"]))
# different course with different filtering (code, unitId)
courses[courses["id"].isin(id_difference)]


,id,code,startDate,endDate,type,name,summary,organizations,organizationId,organizationName,credits,courseUnitId,languageOfInstructionCodes,teachers,enrolmentStartDate,enrolmentEndDate,mincredits,studySubGroups
518,aalto-CUR-164698-3085405,MS-A0001,2023-02-24,2023-02-24,exam-exam,"{'fi': 'Matrix Algebra, Tentti', 'sv': 'Matrix...","{'workload': {'fi': '24&#43;24 (4&#43;4)', 'sv...",[{'organisationId': 'aalto-a3b50b1e-ed7d-4136-...,T302,{'fi': 'Matematiikan ja systeemianalyysin lait...,"{'min': 5, 'max': 5}",aalto-OPINKOHD-1112895518-20210801,[en],[Harri Heimo Petteri Hakula],2022-12-26,2023-02-17,5,"[{'id': 'aalto-ssg-164698-3144-5-11111', 'name..."
606,aalto-CUR-165469-3086176,MS-C1541,2023-01-09,2023-02-23,teaching-participation-lectures,"{'fi': 'Metric Spaces, Luento-opetus', 'sv': '...","{'workload': {'fi': '24&#43;12 (4&#43;2).', 's...",[{'organisationId': 'aalto-a3b50b1e-ed7d-4136-...,T302,{'fi': 'Matematiikan ja systeemianalyysin lait...,"{'min': 5, 'max': 5}",aalto-OPINKOHD-1142267517-20210801,[en],[Kalle Perttu Juhana Kytölä],2022-12-12,2023-01-02,5,"[{'id': 'aalto-ssg-3145-165460-1-01', 'name': ..."
622,aalto-CUR-165738-3086445,PHYS-E0525,2023-04-18,2023-04-18,exam-exam,"{'fi': 'Microscopy of Nanomaterials D, Tentti'...","{'workload': {'fi': '', 'sv': '', 'en': ''}, '...",[{'organisationId': 'aalto-7b8d166f-e80e-4481-...,T304,"{'fi': 'Teknillisen fysiikan laitos', 'sv': 'I...","{'min': 5, 'max': 5}",otm-ebad0f6d-8023-47e2-bfe1-724ca4b10653,[en],"[Janne Tapio Ruokolainen, Hua Jiang]",2023-02-17,2023-04-11,5,"[{'id': 'aalto-ssg-165738-3170-5-11111', 'name..."
629,aalto-CUR-165886-3086593,PHYS-C0256,2023-02-22,2023-02-22,exam-exam,{'fi': 'Thermodynamics and Statistical Physics...,"{'workload': {'fi': '', 'sv': '', 'en': ''}, '...",[{'organisationId': 'aalto-7b8d166f-e80e-4481-...,T304,"{'fi': 'Teknillisen fysiikan laitos', 'sv': 'I...","{'min': 5, 'max': 5}",otm-4cb03484-428b-4232-8db0-16dfff5ed692,[en],[Jukka Pekka Pekola],2022-12-24,2023-02-15,5,"[{'id': 'aalto-ssg-165886-3169-5-11111', 'name..."
633,aalto-CUR-165900-3086607,PHYS-E055103,2023-04-18,2023-04-18,exam-exam,"{'fi': 'Low Temperature Physics D, Superconduc...","{'workload': {'fi': '', 'sv': '', 'en': ''}, '...",[{'organisationId': 'aalto-7b8d166f-e80e-4481-...,T304,"{'fi': 'Teknillisen fysiikan laitos', 'sv': 'I...","{'min': 5, 'max': 6}",otm-38943da8-7b1d-408a-855f-201f0fea96e8,[en],[Vladimir Eltsov],2023-02-17,2023-04-11,5,"[{'id': 'aalto-ssg-165900-3170-5-11111', 'name..."
736,aalto-CUR-166588-3087295,NBE-E4100,2023-05-31,2023-05-31,exam-exam,"{'fi': 'Molecular Biophysics D, Tentti', 'sv':...","{'workload': {'fi': '', 'sv': '', 'en': ''}, '...",[{'organisationId': 'aalto-1311f55a-509b-485d-...,T314,{'fi': 'Neurotieteen ja lääketieteellisen tekn...,"{'min': 5, 'max': 5}",otm-9881eb6c-844e-4431-a1cf-8327dca88257,[en],[Anton Kuzyk],2023-04-01,2023-05-24,5,"[{'id': 'aalto-ssg-166588-3199-5-11111', 'name..."
1272,aalto-CUR-170151-2411704,PHYS-C0252,2023-06-08,2023-06-08,exam-exam,"{'fi': 'Quantum Mechanics, Tentti', 'sv': 'Qua...","{'workload': {'fi': '', 'sv': '', 'en': ''}, '...",[{'organisationId': 'aalto-7b8d166f-e80e-4481-...,T304,"{'fi': 'Teknillisen fysiikan laitos', 'sv': 'I...","{'min': 5, 'max': 5}",otm-7cef259b-e7cd-4495-a080-01c0d58cf8cc,[en],"[Mikko Pentti Matias Möttönen, Tapio Ala-Nissilä]",2023-04-09,2023-06-01,5,"[{'id': 'aalto-ssg-170151-3382-5-11111', 'name..."
1275,aalto-CUR-170160-2412218,PHYS-C0254,2023-04-20,2023-04-20,exam-exam,"{'fi': 'Quantum Circuits, Tentti', 'sv': 'Quan...","{'workload': {'fi': '', 'sv': '', 'en': ''}, '...",[{'organisationId': 'aalto-7b8d166f-e80e-4481-...,T304,"{'fi': 'Teknillisen fysiikan laitos', 'sv': 'I...","{'min': 5, 'max': 5}",otm-54fd870b-cc18-4a73-9bee-eeb5acf2b0d0,[en],"[Gheorghe-Sorin Paraoanu, Jan Goetz]",2023-02-19,2023-04-13,5,"[{'id': 'aalto-ssg-170160-3167-5-11111', 'name..."


In [104]:
MS_C1541_exams = courses[(courses["code"] == "MS-C1541") & (courses["type"] == "exam-exam")]["summary"]
MS_C1541_teachings = courses[(courses["code"] == "MS-C1541") & (courses["type"] == "teaching-participation-lectures")]["summary"]

# exam couese contents
print([item[1]["content"]["en"] for item in MS_C1541_exams.iteritems()])

print("\n")

# teaching couese contents
print(MS_C1541_teachings[606]["content"]["en"])

['real numbers, metric, norm, inner product, open and closed sets, continuous mappings, sequences and limits, compactness, completeness, connectedness.', 'real numbers, metric, norm, inner product, open and closed sets, continuous mappings, sequences and limits, compactness, completeness, connectedness.', 'real numbers, metric, norm, inner product, open and closed sets, continuous mappings, sequences and limits, compactness, completeness, connectedness.', 'real numbers, metric, norm, inner product, open and closed sets, continuous mappings, sequences and limits, compactness, completeness, connectedness.', 'real numbers, metric, norm, inner product, open and closed sets, continuous mappings, sequences and limits, compactness, completeness, connectedness.', 'real numbers, metric, norm, inner product, open and closed sets, continuous mappings, sequences and limits, compactness, completeness, connectedness.']


real numbers, metric, norm, inner product, open and closed sets, continuous map

In [45]:
[course["summary"]["content"]["en"].split() for course in en_data]


228

In [106]:
len(courses["id"].unique())

1405

# BIG ACHEIEVEMENT: FILTER WITH CODE !!!!!!!!!!!

# ANOTHER BIG ACHEIEVEMENT: USE LEARNING OUTCOME AND CONTENT COMBINED !!!!!!!!!!!

In [115]:
unique_code

,id,code,startDate,endDate,type,name,summary,organizations,organizationId,organizationName,credits,courseUnitId,languageOfInstructionCodes,teachers,enrolmentStartDate,enrolmentEndDate,mincredits,studySubGroups
0,aalto-CUR-162063-3082770,LC-1117,2023-01-11,2023-02-15,teaching-participation-lectures,"{'fi': 'Integrated Oral and Written Skills (o,...",{'workload': {'fi': '81 h (Small group instruc...,[{'organisationId': 'aalto-52ed67c2-4791-4ee6-...,U926,"{'fi': 'Aalto University, Kielikeskus', 'sv': ...","{'min': 3, 'max': 3}",aalto-OPINKOHD-1117673055-20210801,[en],[Hanna Liisa Hakala],2022-12-12,2023-01-02,3,"[{'id': 'aalto-ssg-162063-3262-7-11111', 'name..."
8,aalto-CUR-162078-3082785,LC-1310,2023-01-13,2023-04-21,teaching-participation-lectures,{'fi': 'Academic Communication for MSc Student...,{'workload': {'fi': '81 h (Small-group instruc...,[{'organisationId': 'aalto-52ed67c2-4791-4ee6-...,U926,"{'fi': 'Aalto University, Kielikeskus', 'sv': ...","{'min': 3, 'max': 3}",aalto-OPINKOHD-1117673220-20210801,[en],[Jan-Mikael Rybicki],2022-12-12,2023-01-12,3,"[{'id': 'aalto-ssg-162078-3262-7-11111', 'name..."
12,aalto-CUR-162115-3082822,LCA-1015,2023-01-09,2023-04-21,teaching-participation-small-group,"{'fi': 'Autonomous Language Learning Path (o,w...","{'workload': {'fi': '81 h', 'sv': '81 h', 'en'...",[{'organisationId': 'aalto-52ed67c2-4791-4ee6-...,U926,"{'fi': 'Aalto University, Kielikeskus', 'sv': ...","{'min': 3, 'max': 3}",aalto-OPINKOHD-1126097189-20210801,[en],[Hanna Liisa Hakala],2022-12-12,2023-01-02,3,"[{'id': 'aalto-ssg-162115-3262-15-11111', 'nam..."
14,aalto-CUR-162118-3082825,LCA-1022,2023-01-11,2023-02-13,teaching-participation-lectures,"{'fi': 'Academic Writing (w), Luento-opetus', ...",{'workload': {'fi': '81 h (Small-group instruc...,[{'organisationId': 'aalto-52ed67c2-4791-4ee6-...,U926,"{'fi': 'Aalto University, Kielikeskus', 'sv': ...","{'min': 3, 'max': 3}",aalto-OPINKOHD-1126098837-20210801,[en],[Matthew Peter Billington],2022-12-12,2023-01-02,3,"[{'id': 'aalto-ssg-162118-3262-7-11111', 'name..."
17,aalto-CUR-162124-3082831,LC-1113,2023-01-09,2023-04-21,teaching-participation-small-group,{'fi': 'Autonomous Language Learning Path (o...,"{'workload': {'fi': '81 h', 'sv': '81 h', 'en'...",[{'organisationId': 'aalto-52ed67c2-4791-4ee6-...,U926,"{'fi': 'Aalto University, Kielikeskus', 'sv': ...","{'min': 3, 'max': 3}",aalto-OPINKOHD-1126098945-20210801,[en],[Hanna Liisa Hakala],2022-12-12,2023-01-02,3,"[{'id': 'aalto-ssg-162124-3262-15-11111', 'nam..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,otm-e8183aa3-e6d7-4d44-952b-fe4c229426d8,CS-E4675,2023-01-09,2023-06-09,teaching-participation-project,"{'fi': 'Full Stack Web Development D, Projekti...",{'workload': {'fi': 'Student workload consists...,[{'organisationId': 'aalto-3b3aa303-843a-4a29-...,T313,"{'fi': 'Tietotekniikan laitos', 'sv': 'Institu...","{'min': 5, 'max': 7}",aalto-CU-1150933422-20220801,[en],[Arto Hellas],2022-12-12,2023-05-29,5,[{'id': 'otm-49918725-eaeb-44ec-9cc9-5e375e86f...
1401,otm-edfae58b-3d21-4fcb-a82b-3c4a789c0751,CS-E400604,2023-01-01,2023-07-31,teaching-participation-project,{'fi': 'Research Experience Project in Compute...,"{'workload': {'fi': '', 'sv': '', 'en': ''}, '...",[{'organisationId': 'aalto-3b3aa303-843a-4a29-...,T313,"{'fi': 'Tietotekniikan laitos', 'sv': 'Institu...","{'min': 5, 'max': 5}",otm-acc4b322-b771-43cc-b734-3121c68c0b93,[en],[Olli Pekka Orponen],2023-01-01,2023-07-17,5,[{'id': 'otm-35ab9da5-c3b0-4dfd-a666-f48ea033b...
1402,otm-ee70c3a2-38e6-472b-9d38-45386b12fb98,TU-CV00011,2023-02-28,2023-04-28,teaching-participation-lectures,"{'fi': 'Thinking Tools, Luento-opetus', 'sv': ...","{'workload': {'fi': '', 'sv': '', 'en': ''}, '...",[{'organisationId': 'aalto-9a94b0a1-836d-4384-...,T307,"{'fi': 'Tuotantotalouden laitos', 'sv': 'Insti...","{'min': 3, 'max': 3}",otm-86a3f11a-6e29-4691-ab8a-19c9fce9fad6,[en],[Lauri Veikko Järvilehto],2023-01-30,2023-03-06,3,[{'id': 'otm-4509660e-7

In [113]:
# [item[1]["content"]["en"] for item in MS_C1541_exams.iteritems()]
[item[1] for item in MS_C1541_teachings.iteritems()]


[{'workload': {'fi': '24&#43;12 (4&#43;2).',
   'sv': '24&#43;12 (4&#43;2).',
   'en': '24&#43;12 (4&#43;2).'},
  'prerequisites': {'fi': 'MS-A01XX', 'sv': 'MS-A01XX', 'en': 'MS-A01XX'},
  'learningOutcomes': {'fi': 'After passing the course the student knows - methods of rigorous reasoning in mathematical analysis - basic topology of inner product spaces, normed spaces, and metric spaces<br />- the notions of limit and continuity<br />- the definitions and fundamental properties of compactness, completeness, and connectedness in metric spaces.',
   'sv': 'After passing the course the student knows - methods of rigorous reasoning in mathematical analysis - basic topology of inner product spaces, normed spaces, and metric spaces- the notions of limit and continuity- the definitions and fundamental properties of compactness, completeness, and connectedness in metric spaces.',
   'en': 'After passing the course the student knows - methods of rigorous reasoning in mathematical analysis - b

In [126]:
example_course = MS_C1541_teachings[606]["content"]["en"]

In [135]:
example_course

'real numbers, metric, norm, inner product, open and closed sets, continuous mappings, sequences and limits, compactness, completeness, connectedness.'

In [131]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [145]:
tokenized_text = tokenizer.tokenize(example_course)
encoded = tokenizer.encode(
                    example_course,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                    # This function also supports truncation and conversion
                    # to pytorch tensors, but we need to do padding, so we
                    # can't use these features :( .
                    #max_length = 128,          # Truncate all sentences.
                    #return_tensors = 'pt',     # Return pytorch tensors.
               )

In [148]:
tokenized_text = ["[CLS]"] + tokenized_text + ["[SEP]"] 

In [150]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

In [152]:
segments_ids = [1] * len(tokenized_text)

In [149]:
tokenized_text

['[CLS]',
 'real',
 'numbers',
 ',',
 'metric',
 ',',
 'norm',
 ',',
 'inner',
 'product',
 ',',
 'open',
 'and',
 'closed',
 'sets',
 ',',
 'continuous',
 'mapping',
 '##s',
 ',',
 'sequences',
 'and',
 'limits',
 ',',
 'compact',
 '##ness',
 ',',
 'complete',
 '##ness',
 ',',
 'connected',
 '##ness',
 '.',
 '[SEP]']

In [151]:
indexed_tokens

[101,
 2613,
 3616,
 1010,
 12046,
 1010,
 13373,
 1010,
 5110,
 4031,
 1010,
 2330,
 1998,
 2701,
 4520,
 1010,
 7142,
 12375,
 2015,
 1010,
 10071,
 1998,
 6537,
 1010,
 9233,
 2791,
 1010,
 3143,
 2791,
 1010,
 4198,
 2791,
 1012,
 102]

In [153]:
import torch

tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [134]:
from transformers import BertModel

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [155]:
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [162]:
len(hidden_states)

13

In [163]:
len(indexed_tokens)

34